In [1]:
# run whole script
%run 03_safe_execution.ipynb

SQL blocked by guardrails: Forbidden SQL operation detected

Query: ```SELECT * FROM orders```
Valid: True
Reason: SQL is safe to execute

Query: WITH t AS (SELECT * FROM orders) SELECT * FROM t
Valid: True
Reason: SQL is safe to execute

Query: DELETE FROM orders
Valid: False
Reason: Forbidden SQL operation detected

Query: DROP TABLE products
Valid: False
Reason: Forbidden SQL operation detected

Query: SELECT * FROM orders; DELETE FROM orders
Valid: False
Reason: Forbidden SQL operation detected

Query: UPDATE orders SET order_dow = 1
Valid: False
Reason: Forbidden SQL operation detected


In [2]:
import yaml
from pathlib import Path
from typing import Tuple, Dict, Any, TypedDict, Optional

from langgraph.graph import StateGraph, END 

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


True

## Database Connection

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path("D:/code/text-to-sql-agent")))

from src.db.db_connection import get_db_connection

conn = get_db_connection()
cursor = conn.cursor()

## Load Schema

In [4]:
SCHEMA_PATH = Path("../src/schema/schema_summary.yaml")

with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    schema_context: Dict[str, Any] = yaml.safe_load(f)

schema_context.keys()


dict_keys(['tables', 'hints', 'common_joins'])

## Initialize LLm

In [5]:
from langchain_openai import ChatOpenAI
import os

def load_llm():
    return ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )

def call_llm(prompt: str) -> str:
    llm = load_llm()
    response = llm.invoke(prompt)
    return response.content.strip()


## Prompts

In [24]:
# Optimized SQL generation prompt

def build_optimized_prompt(
    question: str,
    schema: Dict[str, Any]
) -> str:
    return f"""
You are an expert PostgreSQL SQL generator.

CRITICAL RULES:
- Output ONLY one SQL SELECT query
- Do NOT include markdown, backticks, or explanations
- Use ONLY tables and columns from the schema
- Follow join templates strictly
- Never invent joins or columns
- Prefer correctness over brevity

Database schema with semantics:
{schema}

User question:
{question}

SQL:
""".strip()


# Optimized correction prompt

def build_optimized_correction_prompt(
    question: str,
    schema: Dict[str, Any],
    previous_sql: str,
    error_reason: str
) -> str:
    return f"""
The SQL query below is INVALID.

Failure reason:
{error_reason}

Rules to fix:
- Use schema exactly as provided
- Follow join templates
- Do not invent columns or tables
- Output ONLY corrected SQL

Schema:
{schema}

Question:
{question}

Invalid SQL:
{previous_sql}

Corrected SQL:
""".strip()

def build_validation_and_response_prompt(
    question: str,
    sql: str,
    results: list
) -> str:
    """
    Creates a prompt for LLM to:
    1. Validate if results answer the question
    2. Generate a natural language response
    """
    # Limit results to first 10 rows for efficiency
    sample_results = results[:10]
    total_rows = len(results)
    
    return f"""
You are a SQL result validator and response generator.

Your tasks:
1. Validate if the SQL query results actually answer the user's question
2. Generate a natural, conversational answer for the user

CRITICAL RULES:
- Output ONLY valid JSON with three fields: "valid" (boolean), "reason" (string), "natural_language_response" (string)
- For validation: Check if query targeted RIGHT tables/columns and results are plausible
- For response: Be conversational, clear, and directly answer the question
- Include relevant numbers/data from results
- Don't mention SQL or technical details in the natural language response
- If results are empty or wrong, explain what went wrong in a user-friendly way

User Question:
{question}

SQL Query Executed:
{sql}

Results (showing {len(sample_results)} of {total_rows} total rows):
{sample_results}

Validation Checklist:
1. Does the SQL query the correct tables/columns for this question?
2. Do the returned values semantically match what was asked?
3. Are the results plausible? (no negative counts, reasonable magnitudes)

Response Guidelines:
- Be direct and conversational
- Use numbers from the results
- Format large numbers readably (e.g., "49,688" not "49688")
- If multiple rows, summarize or show top results
- Don't say "according to the query" - just answer

Output JSON format:
{{
    "valid": true/false,
    "reason": "brief explanation of validation decision",
    "natural_language_response": "conversational answer to user's question"
}}

JSON:
""".strip()


In [7]:
# # -------------------------------------------------------
# # Generate SQL (optimized)
# # -------------------------------------------------------

def generate_sql_optimized(question: str) -> str:
    prompt = build_optimized_prompt(question, schema_context)
    raw_sql = call_llm(prompt)
    return raw_sql


In [25]:
class SQLAgentState(TypedDict):
    question: str
    sql: Optional[str]
    valid: bool
    reason: Optional[str]
    retries: int
    executed: bool          
    results: Optional[list]
    nl_response: Optional[str]

In [9]:
def generate_sql_node(state: SQLAgentState) -> SQLAgentState:
    print("🔄 Generating SQL...")
    raw_sql = generate_sql_optimized(state["question"])
    sql = clean_sql(raw_sql)
    print(f"Generated: {sql[:100]}...")
    return {**state, "sql": sql}


In [10]:
def validate_sql_node(state: SQLAgentState) -> SQLAgentState:
    print("🔍 Validating syntax...")
    is_valid, reason = validate_sql(state["sql"])
    print(f"Syntax valid: {is_valid}")
    if not is_valid:
        print(f"Reason: {reason}")
    return {**state, "valid": is_valid, "reason": None if is_valid else reason, "executed": False}

In [11]:
def execute_sql_node(state: SQLAgentState) -> SQLAgentState:
    print("⚡ Executing SQL...")
    try:
        cursor.execute(state["sql"])
        results = cursor.fetchall()
        print(f"✅ Executed! Got {len(results)} rows")
        return {**state, "executed": True, "results": results, "reason": None}
    except Exception as e:
        print(f"❌ Execution failed: {str(e)[:100]}")
        return {**state, "executed": False, "results": None, "reason": f"Execution error: {str(e)}"}


In [12]:
def validate_execution_node(state: SQLAgentState) -> SQLAgentState:
    print("🔍 Validating execution...")
    if not state["executed"]:
        print("Failed - execution error")
        return {**state, "valid": False}
    if not state["results"] or len(state["results"]) == 0:
        print("Failed - no results")
        return {**state, "valid": False, "reason": "Query returned no results"}
    print("✅ Validation passed!")
    return {**state, "valid": True, "reason": None}

In [13]:
def correct_sql_node(state: SQLAgentState) -> SQLAgentState:
    print(f"🔧 Correcting SQL (retry {state['retries'] + 1}/{MAX_RETRIES})...")
    prompt = build_optimized_correction_prompt(
        question=state["question"],
        schema=schema_context,
        previous_sql=state["sql"],
        error_reason=state["reason"]
    )
    corrected_sql = call_llm(prompt)
    sql = clean_sql(corrected_sql)
    print(f"Corrected: {sql[:100]}...")
    return {**state, "sql": sql, "retries": state["retries"] + 1}

In [26]:
def validate_answer_and_generate_response_node(state: SQLAgentState) -> SQLAgentState:
    """
    Validates if SQL results answer the question AND generates natural language response.
    Does both in a single LLM call for efficiency.
    """
    print("🔍 Validating answer + generating response...")
    
    if not state["executed"] or not state["results"]:
        print("⚠️ Cannot validate - no results to check")
        return {
            **state, 
            "valid": False, 
            "reason": "No results to validate",
            "nl_response": "I couldn't execute the query to get an answer."
        }
    
    # Build combined validation + NL generation prompt
    prompt = build_validation_and_response_prompt(
        question=state["question"],
        sql=state["sql"],
        results=state["results"]
    )
    
    # Get LLM judgment
    response = call_llm(prompt)
    
    # Parse response (expecting JSON format)
    try:
        import json
        output = json.loads(response)
        
        is_valid = output.get("valid", False)
        reason = output.get("reason", "Unknown validation failure")
        nl_response = output.get("natural_language_response", "Unable to generate response.")
        
        if is_valid:
            print("✅ Answer validated! Generated NL response.")
            print(f"📝 Response: {nl_response[:100]}...")
        else:
            print(f"❌ Answer validation failed: {reason}")
        
        return {
            **state, 
            "valid": is_valid, 
            "reason": None if is_valid else reason,
            "nl_response": nl_response
        }
    
    except json.JSONDecodeError as e:
        print(f"⚠️ Failed to parse validation response: {e}")
        return {
            **state, 
            "valid": False, 
            "reason": "Validation parsing error",
            "nl_response": "Error processing the query results."
        }

In [27]:
MAX_RETRIES = 2

def route_after_syntax_check(state: SQLAgentState):
    """Route after pre-execution validation"""
    if state["valid"]:
        return "execute_sql"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"

def route_after_execution(state: SQLAgentState):
    """Route after execution - goes to combined validation+response node"""
    if state["executed"]:
        return "validate_and_respond"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"

def route_after_validation(state: SQLAgentState):
    """Route after validation+response generation"""
    if state["valid"]:
        return END  # Success!
    if state["retries"] >= MAX_RETRIES:
        return END  # Give up
    return "correct_sql"

In [28]:
graph = StateGraph(SQLAgentState)

# Nodes
graph.add_node("generate_sql", generate_sql_node)
graph.add_node("validate_sql", validate_sql_node)
graph.add_node("execute_sql", execute_sql_node)
graph.add_node("validate_and_respond", validate_answer_and_generate_response_node)  # RENAMED + COMBINED
graph.add_node("correct_sql", correct_sql_node)

# Entry point
graph.set_entry_point("generate_sql")

# Flow
graph.add_edge("generate_sql", "validate_sql")

graph.add_conditional_edges(
    "validate_sql",
    route_after_syntax_check,
    {
        "execute_sql": "execute_sql",
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_conditional_edges(
    "execute_sql",
    route_after_execution,
    {
        "validate_and_respond": "validate_and_respond",  # UPDATED
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_conditional_edges(
    "validate_and_respond",  # UPDATED
    route_after_validation,
    {
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_edge("correct_sql", "validate_sql")

# Compile
sql_agent = graph.compile()

In [31]:
initial_state: SQLAgentState = {
    # "question": "how many products are there?", 
    "question": "Show me the top 3 most ordered products",
    "sql": None,
    "valid": False,
    "reason": None,
    "retries": 0,
    "executed": False,
    "results": None,
    "nl_response": None  # NEW
}

final_state = sql_agent.invoke(initial_state)

# Enhanced output with natural language response
print("\n" + "="*60)
if final_state["valid"] and final_state["executed"]:
    print("✅ SUCCESS")
    print("="*60)
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n💬 Answer: {final_state['nl_response']}")
    print(f"\n🔧 SQL: {final_state['sql']}")
    print(f"\n📊 Raw Results: {final_state['results'][:3]}")
else:
    print("❌ FAILED")
    print("="*60)
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n⚠️ Error: {final_state['reason']}")
    print(f"\n💬 Response: {final_state['nl_response']}")
    print(f"\n🔄 Retries: {final_state['retries']}/{MAX_RETRIES}")
    if final_state['sql']:
        print(f"\n🔧 Last SQL: {final_state['sql']}")

print("="*60)


🔄 Generating SQL...
Generated: select p.product_id, p.product_name, count(*) as order_count from order_products_prior op join produ...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
✅ Executed! Got 3 rows
🔍 Validating answer + generating response...
✅ Answer validated! Generated NL response.
📝 Response: The top three most ordered products are Banana with 472,565 orders, Bag of Organic Bananas with 379,...

✅ SUCCESS

🙋 Question: Show me the top 3 most ordered products

💬 Answer: The top three most ordered products are Banana with 472,565 orders, Bag of Organic Bananas with 379,450 orders, and Organic Strawberries with 264,683 orders.

🔧 SQL: select p.product_id, p.product_name, count(*) as order_count from order_products_prior op join products p on op.product_id = p.product_id group by p.product_id, p.product_name order by order_count desc limit 3

📊 Raw Results: [(24852, 'Banana', 472565), (13176, 'Bag of Organic Bananas', 379450), (21137, 'Organic Strawberries', 264683)